In [2]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

news=[]
url='https://news.naver.com/'
response = urllib.request.urlopen(url)
soup=BeautifulSoup(response,'html.parser')
results=soup.select('.section_list_ranking li a')
for result in results:
    #print('제목:',result.attrs['title'])
    #print('링크:',result.attrs['href'])
    title=result.attrs['title']
    href=result.attrs['href']
    url_content='https://news.naver.com'+result.attrs['href']
    response_content = urllib.request.urlopen(url_content)
    soup_content=BeautifulSoup(response_content,'html.parser')
    content=soup_content.select_one('#articleBodyContents')
    #print(content.contents)

    #가공작업
    output=''
    for item in content.strings:
        stripped = str(item).strip()    #공백제거
        if stripped=='':
            continue
        else:
             output+=item
    output=output.replace('본문 내용TV플레이어','')
    output=output.replace('// flash 오류를 우회하기 위한 함수 추가','')
    output=output.replace('function _flash_removeCallback() {}','')
    output=output.strip()
    news.append({'title':title,'href':href,'contents':output})
    
#print(news)    
dataframe=pd.DataFrame(news)
dataframe.to_csv("news.csv")

In [3]:
import pandas as pd
df = pd.read_csv('news.csv', engine = 'python', encoding = 'utf-8')

In [4]:
df.shape

(60, 4)

In [5]:
import re
p = '.*(코로나|여행|비행기|호텔).*' # . 임의의 문자 한개에 대응, * 0개 이상
topic = df[df['title'].str.match(p)|df['contents'].str.match(p, flags = re.MULTILINE)]
topic.shape

(40, 4)

In [6]:
topic.head() # topic.tail() topic.sample(3)

,Unnamed: 0,title,href,contents
1,1,"[단독]곽상도 ""마스크 독점 지오영, 한달 204억 벌었다"" 주장",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,곽상도 미래통합당 의원은 5일 정부의 시중 약국 대상 공적(公的) 마스크 공급 채널...
2,2,"통합당, 조국 고리로 與 때리기…""'조국 살리자' 선거전 기이해""(종합)",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,"김종인 ""조국과 대한민국 경제 중 무엇을 살려야 할지 삼척동자도 알아""원유철·대전 ..."
3,3,"문 대통령 ""강원 산불 이겨낸 정신으로 코로나19도 이겨낼 것""(종합)",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,"식목일 맞아 산불 피해현장에 금강송 심어…""강원산불, 재난극복 모범적 사례""""코로나..."
4,4,"文대통령 ""코로나에도 전국민 1인1나무 가꾸기 운동 당부""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,"文대통령 부부, 식목일에 강릉 산불 피해지 방문피해 주민 등 40여명과 금강소나무 ..."
5,5,"김종인 ""경제 바이러스가 온다… 조국과 경제, 둘중 뭘 살릴거냐""",/main/ranking/read.nhn?mid=etc&sid1=111&rankin...,"대전 찾은 김종인 ""민주당 후보, 거수기 노릇""김종인 미래통합당 총괄선대위원장이 5..."


# 워드클라우드 작성 함수

In [17]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

def displayWordCloud(data = None, backgroundcolor = 'white', width = 800, height = 600):
    wordcloud = WordCloud(
                        # font_path = '/Library/Fonts/NanumBarunGothic.ttf',
                        font_path = '/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/NanumPen.ttf',
                        stopwords = stopwords_kr,
                        background_color = backgroundcolor,
                        width = width, height = height).generate(data)
    # print(wordcloud.words_)
    plt.figure(figsize = (15, 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [18]:
%time displayWordCloud(' '.join(df['contents']))

UnboundLocalError: local variable 'stopwords_kr' referenced before assignment

In [19]:
from soynlp.noun import LRNounExtractor

In [20]:
%%time
noun_extractor = LRNounExtractor(verbose = True)
noun_extractor.train(df['contents'])
nouns = noun_extractor.extract()

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (2245, 1238) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 545 nouns are extracted
CPU times: user 328 ms, sys: 3.8 ms, total: 332 ms
Wall time: 334 ms


In [21]:
%time displayWordCloud(' '.join(nouns))

UnboundLocalError: local variable 'stopwords_kr' referenced before assignment